In [1]:
! echo 'PINECONE_API_KEY=your-api-key' >> .env 

In [4]:
! pip install dotenv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
        Using cached distribute-0.7.3.zip (145 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        Ã— Preparing metadata (pyproject.toml) did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [6 lines of output]
            usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
               or: setup.py --help [cmd1 cmd2 ...]
               or: setup.py --help-commands
               or: setup.py cmd --help
      
            

In [6]:
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [7]:
! pip install pinecone-client

In [10]:
from pinecone import Pinecone

# import os
# key = os.getenv("PINECONE_API_KEY")

key = '280907f0-3169-4914-90d4-7b694a914e93' 

client = Pinecone(api_key=key)

client.describe_index('index1')

{'dimension': 1024,
 'host': 'index1-ku37u4h.svc.gcp-starter.pinecone.io',
 'metric': 'dotproduct',
 'name': 'index1',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [ ]:
from pinecone import PodSpec
# run it when get fucking credits
client.create_index(
    name='index2',
    dimension=1024,
    spec = PodSpec(
        environment='gcp-starter'
    )
)

In [12]:
index = client.Index('index1')

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('thenlper/gte-large')


In [14]:
random_sentences = [
    "The sunsets over the horizon, painting the sky in vibrant hues of orange and pink.",
    "Coffee beans release a rich aroma as they grind, signaling the start of a new day.",
    "In the forest, the leaves rustle gently in the breeze, creating a soothing melody.",
    "Technology continues to evolve, shaping the way we communicate and interact with the world !",
    "Lost in a good book, the reader becomes immersed in a world of imagination and adventure.",
    "Laughter echoed through the air as friends gathered around the bonfire on a chilly evening.",
    "The old lighthouse stood tall, its beacon guiding ships safely through the dark night.",
    "Raindrops danced on the window pane, creating a rhythmic melody on a gloomy afternoon.",
    "As the first snowflake fell, the town transformed into a winter wonderland.",
    "A distant train whistle evoked a sense of nostalgia, echoing through the quiet countryside.",
    "Gazing at the stars, one can't help but marvel at the vastness of the universe.",
    "The aroma of freshly baked bread wafted through the bakery, enticing passersby.",
    "Children's laughter filled the playground as they played games and shared stories.",
    "A solitary figure walked along the beach, lost in thought, as waves gently lapped the shore.",
    "The puzzle pieces clicked together, revealing a completed picture and a sense of accomplishment.",
]


In [15]:
embeddings = [model.encode(i) for i in random_sentences]

In [16]:
vectors = []

for i in range(len(random_sentences)):
    vectors.append({'id': str(i), 'values': embeddings[i], 'metadata' : {'text': random_sentences[i]}})


vectors[1]

{'id': '1',
 'values': array([-0.00900913,  0.01461186, -0.0134896 , ...,  0.0049426 ,
        -0.02139149,  0.01867538], dtype=float32),
 'metadata': {'text': 'Coffee beans release a rich aroma as they grind, signaling the start of a new day.'}}

In [17]:
index.upsert(
    vectors= vectors,
    namespace='sample-namespace'

)

{'upserted_count': 15}

In [18]:
query = 'old lighthouse stood tall'

query_vector = model.encode(query)

In [24]:
result = index.query(
    vector=query_vector.tolist(),
    top_k=2,
    namespace='sample-namespace',
    include_values=True,
    include_metadata=True
   
)

# result

result['matches'][0].metadata['text']


'The old lighthouse stood tall, its beacon guiding ships safely through the dark night.'

In [25]:
index.delete(
    ids=['1','2'],
    namespace='sample-namespace'    
)

index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0003,
 'namespaces': {'sample-namespace': {'vector_count': 15},
                'sample_namspace': {'vector_count': 15}},
 'total_vector_count': 30}

Doing it with csv file

In [26]:
! pip install pandas

In [27]:
import pandas as pd 

df = pd.read_csv('menu_items.csv')

df['item_name']

C:\Users\olive\AppData\Local\Temp\ipykernel_18732\471407655.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


0                       Vegan Chicken Salad
1                   Spring Rolls (4 pieces)
2                   Pot Stickers (6 pieces)
3                  Fried Wonton (10 pieces)
4                 Fried Tofu with Soy Sauce
                       ...                 
196                       Fried Turnip Ball
197                              Fried Curd
198                        Stuffed Eggplant
199         Black Pepper Vegan Chicken Roll
200    Crispy Vegan Chicken with Mixed Nuts
Name: item_name, Length: 201, dtype: object

In [28]:
item_ids = df['item_id']
item_names = df['item_name']

In [29]:
embeds = [model.encode(i) for i in item_names]

vectors = []

for i in range(len(item_ids)):
    vectors.append({'id': str(i), 'values': embeds[i], 'metadata' : {'item_name': item_names[i], 'item_id': item_ids[i]}})


vectors[1]

{'id': '1',
 'values': array([-0.02061882, -0.01435536, -0.01261128, ..., -0.02458902,
         0.00575563, -0.00873372], dtype=float32),
 'metadata': {'item_name': 'Spring Rolls (4 pieces)', 'item_id': 'A2'}}

In [30]:
index.upsert(
    vectors=vectors,
    namespace='menu_items'
)

{'upserted_count': 201}

In [31]:
query = 'Vegan'

query_vector = model.encode(query)

In [35]:
result = index.query(
    vector= query_vector.tolist(),
    namespace='menu_items',
    include_values=True,
    include_metadata=True,
    top_k=2,
)

result

{'matches': [{'id': '120',
              'metadata': {'item_id': 'L11',
                           'item_name': 'Vegan Pork and Cabbage'},
              'score': 0.874096215,
              'values': [-0.0159485843,
                         0.00176796899,
                         -0.0032066959,
                         0.0195717402,
                         -0.00950619485,
                         -0.00331995101,
                         -0.00401564734,
                         0.0566319823,
                         0.00907088909,
                         0.00783966854,
                         0.0347831398,
                         -0.0229080096,
                         0.00554968975,
                         -0.0306092426,
                         -0.00923932809,
                         -0.034360297,
                         -0.00078283035,
                         -0.0250529293,
                         -0.0438567661,
                         -0.00978260487,
                       